In [1]:
import sys
import pandas as pd
from datetime import date
import os

In [34]:
def update_date_df(dataframe, curret_date:str=date.today()) -> str:
    datevar = curret_date
    dataframe['Fecha de actualizacion'] = datevar
    return datevar.__str__().replace('-', '.')

In [35]:
def create_base_prices_list(path_file:str, margin:int=0.3):
    """La lista de de precios inicial debe tener los siguientes campos: 'Codigo de origen', 'Descripcion Producto', 'Master Pack', 'PVP'.
    El margen por defectdo es 30%, lo cual significa que la lista de precios base se generará en base a ese margen base."""
    df = pd.read_excel(path_file)
    df['30%'] = df['PVP']*(1-margin)
    datestr = update_date_df(df, '11-11-2022')
    df.to_excel('result.xlsx', index=False, sheet_name=datestr)
  #  os.system("result.xlsx")

In [36]:
def update_prices(base_list_prices_path:str, new_list_prices_path:str):
    df_base = pd.read_excel(base_list_prices_path)
    new_df = pd.read_excel(new_list_prices_path)
    new_df['30%'] = new_df['PVP']*0.3
    datastr=update_date_df(new_df, '01-02-2023')
    updated_prices_list = pd.concat([df_base, new_df], ignore_index=True, axis=0).drop_duplicates(keep='last', subset=['COD. ORIGEN'])
    updated_prices_list.to_excel('result.xlsx', index=False, sheet_name=datastr)

In [77]:
base = pd.read_excel("LISTA_BASE.xlsx", index_col=None)
base

,COD. ORIGEN,DESCRIPCION,MASTER PACK,PVP
0,1416228,"Teamsterz autos de metal 3"" en blister",48,6.9
1,1416576,Teamsterz lanzador ataque dinosaurio incl 3 a...,12,39.9
2,1416803,Teamsterz camión para guardar 12 autos incl 4 ...,12,59.9
3,1416211,"Teamsterz autos de metal 3"" pack de 2",18,13.8
4,1416849,Teamsterz lanzador con caldero slime incl 1 au...,12,39.9
...,...,...,...,...
490,54334,Cubo de actividades de ensueño,1,279.0
491,54723,Caja piramide actividades multiples,1,199.0
492,330721,GOLIATH - MANDÍBULAS JUEGO DE PESCA,6,109.0
493,30546,GOLIATH - ANTON EL CERDITO COMELÓN,NaN,119.0


### Validación para detectar registros que no tiene COD. ORIGEN

In [83]:
base[base["COD. ORIGEN"].isnull()]

,COD. ORIGEN,DESCRIPCION,MASTER PACK,PVP


In [80]:
len(base[base["COD. ORIGEN"].isnull()].index)

0

### Validación para detectar registros del mismo SKU repetidos
En la lista de precios base no deben existir más de un registro por SKU, en otras palabras, no deberían existir dos registros con el mismo codigo de origen **(COD. ORIGEN)**.

In [89]:
df = base.groupby(["COD. ORIGEN"], as_index=False).size()
df[df['size'] != 1]

,COD. ORIGEN,size


In [51]:
df = base.groupby(["COD. ORIGEN"], as_index=False).size()
len(df[df['size'] != 1].index)

0

### Cargamos la nueva lista de precios


In [91]:
nueva_lista = pd.read_excel('LISTA_NUEVA.xlsx')
nueva_lista

,COD. ORIGEN,DESCRIPCION,MASTER PACK,PVP
0,70902,AIR STUNT SHOW: AVIONETA DE TIGRE,4,159.0
1,70833,AIR STUNT SHOW HELICÓPTERO CON EQUIPO DE FILMA...,4,239.0
2,70550,ESPECTÁCULO DE ACROBACIAS CAMIÓN MONSTRUO TIBURÓN,2,469.0
3,70831,ESPECTÁCULO DE ACROBACIAS AÉREAS BIPLANO PHOENIX,4,489.0
4,70832,ESPECTÁCULO DE ACROBACIAS AÉREAS EAGLE JET,4,549.0
...,...,...,...,...
188,70817,STARTER PACK POLICÍA: ENTRENAMIENTO,4,139.0
189,70820,STARTER PACK STUNTSHOW QUAD CON RAMPA DE FUEGO,4,179.0
190,70009,PLAYMOBIL SUPERSET GALAXIA,4,139.0
191,70645,LLAVERO A-TEAM B.A.,10,39.9


In [95]:
nueva_lista_tmp = pd.concat([base, nueva_lista])
nueva_lista_tmp

,COD. ORIGEN,DESCRIPCION,MASTER PACK,PVP
0,1416228,"Teamsterz autos de metal 3"" en blister",48,6.9
1,1416576,Teamsterz lanzador ataque dinosaurio incl 3 a...,12,39.9
2,1416803,Teamsterz camión para guardar 12 autos incl 4 ...,12,59.9
3,1416211,"Teamsterz autos de metal 3"" pack de 2",18,13.8
4,1416849,Teamsterz lanzador con caldero slime incl 1 au...,12,39.9
...,...,...,...,...
188,70817,STARTER PACK POLICÍA: ENTRENAMIENTO,4,139.0
189,70820,STARTER PACK STUNTSHOW QUAD CON RAMPA DE FUEGO,4,179.0
190,70009,PLAYMOBIL SUPERSET GALAXIA,4,139.0
191,70645,LLAVERO A-TEAM B.A.,10,39.9


In [107]:
t = nueva_lista_tmp.groupby(['COD. ORIGEN'], as_index=False).size()
t[t['size'] > 1]

,COD. ORIGEN,size
22,5649,2
89,9461,2
98,10127,2
131,70009,2
135,70033,2
...,...,...
355,70930,2
373,71090,2
374,71091,2
375,71092,2
